In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('diabetes.csv')

In [3]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
X = df.drop(['Outcome'], axis =1)
y = df.Outcome

In [5]:
df.Outcome.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_sca = scaler.fit_transform(X)
X_sca

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [7]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X_sca, y, stratify = y, random_state = 10)

In [8]:
len(X_train)

576

In [9]:
y_train.value_counts()

0    375
1    201
Name: Outcome, dtype: int64

In [10]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
from sklearn.model_selection import cross_val_score as cvs
score = cvs(model, X, y, cv =5)
score.mean()

0.7188778541719717

In [11]:
from sklearn.ensemble import BaggingClassifier
bg = BaggingClassifier(
                base_estimator= model,
                n_estimators = 100, # -> it means the no. of subsets from the samples you wanna train paralelly
                max_samples=0.8, # ->using 80% out of all sample to train the model
                oob_score=True, # ->(out of bag score) after taking 80% data to train my model from x_train from which we are making
                                # 100 subsets of it by which we will take majority vote and make final decission, but we might
                                # still miss some data from the 80% data to train because the subsets are picked very randomly
                                # it doest track what its already picked it just kepp picking randomly so according to probability
                                # it might not pick some samples for any of the 100 subsets, so that data we can use test and see
                                # the score so that score is called oob_score
                random_state=0) # -> its for predictability 
bg.fit(X_train,y_train)
bg.oob_score_

0.7534722222222222

In [12]:
bg.score(X_test, y_test)

0.7760416666666666

In [13]:
score = cvs(bg, X, y, cv = 5)
score.mean()

0.7578728461081402